In [2]:
##############################################
# Programmer: Matthew Moore
# Class: CptS 322-01, Spring 2022
# Programming Assignment #4
# 3/3/2022
# 
# 
# Description: This program runs through all of the visualizations for the assignment. The purpose of the file is
# to test the predictive accuracy of all of the classification methods we build in the myclassifiers.py class file. 
##############################################
import importlib
import os
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
import mysklearn.myevaluation as myeval
import mysklearn.mysimplelinearregressor
from mysklearn.mysimplelinearregressor import MySimpleLinearRegressor
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MySimpleLinearRegressionClassifier,\
    MyKNeighborsClassifier,\
    MyDummyClassifier,\
    MyDecisionTreeClassifier

In [7]:
importlib.reload(np)
importlib.reload(os)
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.myevaluation)
np.random.seed(0)

fname = os.path.join("input_data", "cbb.csv")
bball_table = MyPyTable()
bball_table.load_from_file(fname)

fname = os.path.join("input_data", "cbb2022.csv")
bball_table_test = MyPyTable()
bball_table_test.load_from_file(fname)
# stats_header = ['ADJOD','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD',\
    # 'ORB','DRB','FTR','FTRD','2P_O','2P_D','3P_O','3P_D','ADJ_T','WAB']

stats_header = ['ADJOE','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD',\
    'DRB','FTR','3P_O','3P_D','ADJ_T','WAB']

stats_cols = []
stats_cols_inner = []
stats_col = []
for stat in stats_header:
    stats_col.append(myutils.discretize(myutils.normalize(bball_table_test.get_column(stat))))
stats_col.append(bball_table_test.get_column('SEED'))

for index in range(len(bball_table_test.data)):
    for stat_col in stats_col:
        stats_cols_inner.append(stat_col[index])
    stats_cols.append(stats_cols_inner)
    stats_cols_inner = []

X_test = stats_cols.copy()
stats_cols = []
stats_col = []
# Grabbing all the rows we want to use
for stat in stats_header:
    stats_col.append(myutils.discretize(myutils.normalize(bball_table.get_column(stat))))
stats_col.append(bball_table.get_column('SEED'))

# Creating a new table with the rows based on the appropriate columns
for index in range(len(bball_table.data)):
    for stat_col in stats_col:
        stats_cols_inner.append(stat_col[index])
    stats_cols.append(stats_cols_inner)
    stats_cols_inner = []

y_train_bball = [val for val in bball_table.get_column('POSTSEASON')]
X_train_bball = stats_cols.copy()
# Step 1: Discretize all of the stats columns into bins
# Step 2: Randomly generate 1/3 of data into a test set and 2/3 into a training set
X_train_bball, X_test_bball, y_train_bball, y_test_bball = myeval.train_test_split(X_train_bball, y_train_bball, test_size=0.33,random_state=10)
# Step 4: Utilize bootstrap method to generate a forest of possible trees for training set
n = 100
tree_ratings = []
my_tree = MyDecisionTreeClassifier()
for i in range(n):
    X_sample, X_out_of_bag, y_sample, y_out_of_bag = myeval.bootstrap_sample(X_train_bball, y_train_bball, n_samples=myutils.normal_round(len(X_train_bball) / 2), random_state=i)
    my_tree.fit(X_sample, y_sample)
    tree_predictions = my_tree.predict(X_out_of_bag)
    tree_ratings.append([i , myeval.accuracy_score(y_out_of_bag, tree_predictions)])
    # TODO: Do we need to randomly create an assortment of available attributes for each of these trees??
    # TODO: Add more instances to the dataset to meet the 1000 requirement
    # TODO: use this link: https://barttorvik.com/trank.php?year=2022&sort=&top=0&conlimit=All# 

for tree_rating in tree_ratings:
    if tree_rating[1] > 0.5:
        print(tree_rating)

# Step 5: 

X_train_practice = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9,10],[10,11]]
y_train_practice = [0,0,0,0,0,1,1,1,1,1]
bootstrap_solution = myeval.bootstrap_sample(X_train_practice,y_train_practice)
# print(bootstrap_solution)
my_tree = MyDecisionTreeClassifier()
my_tree.fit(X_train_bball,y_train_bball)
# print(my_tree.predict(X_test))


[26, 0.517948717948718]
[33, 0.518918918918919]
[64, 0.5487179487179488]
[90, 0.5026455026455027]
[99, 0.5440414507772021]


In [4]:
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.mysimplelinearregressor)
k_values = [1,2,3,4,5,6,7,8,9,10,11,12,13]
print("===========================================")
print("STEP 2: Nearest Neighbor MPG Classifier")
print("===========================================")
results = ["Champions","2ND","F4","E8","S16","R32","R64","R68"]
parallel_predictions = []
for k in k_values:
    knn_classifier = MyKNeighborsClassifier(k)
    y_train_auto = bball_table.get_column('POSTSEASON')
    knn_classifier.fit(X_train_bball,y_train_bball)
    y_predictions = knn_classifier.predict(X_test)
    teams = bball_table_test.get_column("TEAM")
    num_correct = 0
    parallel_predictions.append(y_predictions)
    for result in results:
        # print(result)
        for i, test_instance in enumerate(X_test):
            # actual_mpgs.append([test_instance[bball_table.column_names.index('POSTSEASON')]])
            if result == y_predictions[i]:
                # print(teams[i],"places as ",y_predictions[i])
                pass

for j, team in enumerate(parallel_predictions[0]):
    print(teams[j],"places as ",end=' ')
    for i, y_predictions in enumerate(parallel_predictions):
        # actual_mpgs.append([test_instance[bball_table.column_names.index('POSTSEASON')]])
        print(results.index(parallel_predictions[i][j]),end=' ')
     
    print("")


STEP 2: Nearest Neighbor MPG Classifier
Gonzaga places as  3 1 3 3 3 3 3 3 3 3 3 3 3 
Kansas places as  5 5 5 5 5 5 5 5 3 5 5 3 3 
Baylor places as  5 5 3 0 5 5 0 0 0 3 3 3 3 
Arizona places as  5 3 3 3 3 3 3 3 3 3 3 3 5 
San Francisco places as  5 5 5 5 5 5 5 5 5 5 5 5 5 
Loyola Chicago places as  5 5 5 5 5 5 5 5 5 5 5 5 5 
Davidson places as  5 5 5 6 6 6 6 6 6 6 6 6 6 
Miami FL places as  6 5 6 5 6 5 6 5 5 5 5 5 6 
Virginia Tech places as  5 5 5 5 5 5 5 5 5 5 5 5 5 
Michigan places as  7 5 5 5 5 5 6 5 6 6 6 6 6 
Notre Dame places as  5 5 5 6 5 5 6 6 6 6 6 6 6 
Iowa St. places as  5 5 5 5 5 5 5 5 5 6 6 6 6 
Rutgers places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Indiana places as  6 6 6 6 6 6 6 6 5 6 6 6 6 
UAB places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Wyoming places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
New Mexico St. places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Richmond places as  6 6 5 6 6 6 6 6 6 6 6 6 6 
Vermont places as  5 5 6 6 6 6 6 6 6 6 6 6 6 
South Dakota St. places as  6 6 6 6 6 6 6 6 6 6 6 6 6 
Cha

In [5]:
importlib.reload(mysklearn.myutils)
importlib.reload(mysklearn.mypytable)
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.mysimplelinearregressor)
# RQ5 (fake) iPhone purchases dataset
iphone_col_names = ["standing", "job_status", "credit_rating", "buys_iphone"]
X_train_iphone = [
    [1, 3, "fair"],
    [1, 3, "excellent"],
    [2, 3, "fair"],
    [2, 2, "fair"],
    [2, 1, "fair"],
    [2, 1, "excellent"],
    [2, 1, "excellent"],
    [1, 2, "fair"],
    [1, 1, "fair"],
    [2, 2, "fair"],
    [1, 2, "excellent"],
    [2, 2, "excellent"],
    [2, 3, "fair"],
    [2, 2, "excellent"],
    [2, 3, "fair"]
]
y_train_iphone = ["no","no","yes","yes","yes","no","yes","no","yes","yes","yes","yes","yes","no","yes"]
my_tree = MyDecisionTreeClassifier()
my_tree.fit(X_train_iphone,y_train_iphone)
print(my_tree.tree)

['Attribute', 'att0', ['Value', 1, ['Attribute', 'att1', ['Value', 1, ['Leaf', 'yes', 1, 5]], ['Value', 2, ['Attribute', 'att2', ['Value', 'excellent', ['Leaf', 'yes', 1, 2]], ['Value', 'fair', ['Leaf', 'no', 1, 2]]]], ['Value', 3, ['Leaf', 'no', 2, 5]]]], ['Value', 2, ['Attribute', 'att2', ['Value', 'excellent', ['Leaf', 'no', 4, 10]], ['Value', 'fair', ['Leaf', 'yes', 6, 10]]]]]
